In [33]:
#!/usr/bin/env python3
# Anchor extraction from HTML document
from bs4 import BeautifulSoup
from urllib.request import urlopen

with urlopen('https://www.naver.com/') as response:
    soup = BeautifulSoup(response, 'html.parser')
    for anchor in soup.select('span.ah_k'):
        print(anchor)
#urlopen 사용하면 매우 편하지만, 네이버 구글 등 막히는 추세임. 때문에 가상 웹브라우저를 사용해야 함.

https://beomi.github.io/gb-crawling/posts/2017-09-28-HowToMakeWebCrawler-Headless-Chrome.html 에서 설명하는, 최종단계

In [32]:
from selenium import webdriver
from bs4 import BeautifulSoup

TEST_URL = 'https://www.naver.com'

###############################################################for headless options
option = webdriver.ChromeOptions()
option.add_argument('headless')
option.add_argument('window-size=1920x1080')
option.add_argument("disable-gpu")
option.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
option.add_argument("lang=ko_KR") # 한국어 플러그인 추가
options.add_argument("proxy-server=localhost:8080") # Proxy server : mitmdump -p 8080 -s "inject.ipynb"
driver=webdriver.Chrome('C:/Users/mod96/Desktop/HSH/Programs/WebCrawling/chromedriver', options=option)
###############################################################

driver.get(TEST_URL)

###############################################################

html = driver.page_source ## 페이지의 elements모두 가져오기
soup = BeautifulSoup(html, 'html.parser') ## BeautifulSoup사용하기
notices = soup.select('span.ah_k')

print(notices)
for n in notices:
    print(n.text.strip())

[]


proxy 안켜도 불러는 와지는듯?

In [78]:
from selenium import webdriver
from bs4 import BeautifulSoup

TEST_URL = 'https://datalab.naver.com/keyword/realtimeList.naver?where=main'

############################################################### for headless options
option = webdriver.ChromeOptions()
option.add_argument('headless')
option.add_argument('window-size=1920x1080')
option.add_argument("disable-gpu")
driver=webdriver.Chrome('C:/Users/mod96/Desktop/HSH/Programs/WebCrawling/chromedriver', options=option)
###############################################################

driver.get(TEST_URL)

###############################################################

html = driver.page_source ## 페이지의 elements모두 가져오기
soup = BeautifulSoup(html, 'html.parser') ## BeautifulSoup사용하기 <class 'bs4.BeautifulSoup'>
hotKeys=soup.select("span.item_title") ## 특정 부분만 가져오기 <class 'bs4.element.ResultSet'> <- select 안먹힘. select_one 써야함.
for num,key in enumerate(hotKeys,1):
    print(num,'.',key.text)

1 . 곽도원
2 . 설국열차
3 . 곽도원 나이
4 . 김대명
5 . 서대문구청
6 . 김희원
7 . 김대명 나이
8 . n번방 사건
9 . 국제수사
10 . 김하영
11 . 유민상 김하영
12 . 도봉구청홈페이지
13 . 환희
14 . 오종혁
15 . 유민상
16 . 이상수
17 . 이태원 클라쓰 16회 예고
18 . 서대문구 확진자
19 . 서대문구 확진자 동선
20 . 전인화 나이


In [65]:
print(type(hotKeys),
type(soup))

<class 'bs4.element.ResultSet'> <class 'bs4.BeautifulSoup'>


주가 불러오기

In [6]:
import requests
from bs4 import BeautifulSoup
 
def get_bs_obj(company_code):
    url = 'https://finance.naver.com/item/main.nhn?code=' + company_code
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, 'html.parser')
    return bs_obj
 
def get_candle_chart(company_code):
    bs_obj = get_bs_obj(company_code)
 
    # close 종가(전일)
    td_first = bs_obj.find('td', {'class': 'first'})  # 태그 td, 속성값 first 찾기
    blind = td_first.find('span', {'class': 'blind'})  # 태그 span, 속성값 blind 찾기
    close = blind.text
 
    # high 고가
    table = bs_obj.find('table', {'class': 'no_info'})  # 태그 table, 속성값 no_info 찾기
    trs = table.find_all('tr')  # tr을 list로 []
    first_tr = trs[0]  # 첫 번째 tr 지정
    tds = first_tr.find_all('td')  # 첫 번째 tr 안에서 td를 list로
    second_tds = tds[1]  # 두 번째 td 지정
    high = second_tds.find('span', {'class': 'blind'}).text
 
    # open 시가
    second_tr = trs[1]  # 두 번째 tr 지정
    tds_second_tr = second_tr.find_all('td')  # 두 번째 tr 안에서 td를 list로
    first_td_in_second_tr = tds_second_tr[0]  # 첫 번째 td 지정
    open = first_td_in_second_tr.find('span', {'class': 'blind'}).text
 
    # low 저가
    second_td_in_second_tr = tds_second_tr[1]  # 두 번째 td 지정
    low = second_td_in_second_tr.find('span', {'class': 'blind'}).text
 
    return {'close': close, 'high': high, 'open': open, 'low': low}
 
# 펄어비스 회사 코드는 ”263750”
# 삼성전자 회사 코드는 ”005930”
# 셀트리온 회사 코드는 ”068270”
company_codes = ['263750', '005930', '068270']
 
for item in company_codes:
    candle_chart = get_candle_chart(item)
    print(candle_chart)

{'close': '161,200', 'high': '169,900', 'open': '163,700', 'low': '163,700'}
{'close': '42,950', 'high': '45,500', 'open': '44,150', 'low': '43,550'}
{'close': '140,000', 'high': '153,000', 'open': '145,500', 'low': '141,000'}


하루치 데이터 불러오기

In [20]:
from selenium import webdriver
from bs4 import BeautifulSoup

############################################################### for headless options
option = webdriver.ChromeOptions()
option.add_argument('headless')
option.add_argument('window-size=1920x1080')
option.add_argument("disable-gpu")
driver=webdriver.Chrome('C:/Users/mod96/Desktop/HSH/Programs/WebCrawling/chromedriver', options=option)
###############################################################
def Oneday(code,date): # 둘다 str 형식.
    Price=[]
    for i in range(39):
        pagenum=str(i+1)
        URL = 'https://finance.naver.com/item/sise_time.nhn?code='+code+'&thistime='+date+'&page='+pagenum
        driver.get(URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ############################################
        #time=soup.select("span.tah.p10.gray03")
        #T=[]
        #for t in time:
        #    T.append(t.text)
        #print(T)
        ############################################  
        main=soup.select("td.num")
        M=[]
        for m in main:
            M.append(m.text)
        k=0
        while 6*k<len(M):
            Price.append(M[6*k])
            k=k+1
        ############################################    
    Price.reverse()
    return Price
            

###############################################################
Price=Oneday('272450','2020032016')
for i in range(len(Price)):
    print(Price[i])

5,500
5,650
5,590
5,580
5,600
5,550
5,500
5,470
5,420
5,440
5,390
5,160
5,290
5,200
5,240
5,200
5,240
5,250
5,250
5,240
5,260
5,240
5,200
5,200
5,240
5,280
5,250
5,270
5,260
5,260
5,250
5,250
5,220
5,220
5,220
5,200
5,170
5,150
5,150
5,170
5,180
5,100
5,090
5,110
5,120
5,100
5,110
5,120
5,130
5,120
5,110
5,100
5,060
5,030
5,020
5,010
5,070
5,060
5,040
5,040
5,030
5,030
5,100
5,140
5,120
5,120
5,110
5,120
5,130
5,170
5,170
5,170
5,160
5,170
5,150
5,180
5,200
5,260
5,250
5,250
5,250
5,200
5,230
5,220
5,220
5,180
5,200
5,210
5,230
5,220
5,240
5,230
5,230
5,230
5,220
5,260
5,270
5,300
5,340
5,280
5,280
5,250
5,260
5,240
5,240
5,250
5,260
5,260
5,280
5,280
5,300
5,280
5,320
5,340
5,360
5,350
5,380
5,380
5,410
5,450
5,550
5,600
5,520
5,600
5,590
5,610
5,580
5,550
5,600
5,600
5,600
5,630
5,640
5,720
5,760
5,780
5,840
5,720
5,780
5,810
5,810
5,880
5,900
5,930
5,920
5,910
5,830
5,800
5,830
5,790
5,750
5,680
5,720
5,660
5,640
5,670
5,680
5,690
5,700
5,690
5,670
5,670
5,670
5,620
5,660
5,620
5,66

Oneday 로 일주일치까지 불러와지기 가능. KOSPI 는 'KOSPI' 임...

<가장 간단한 러닝을 사용하기 위해 디자일을 해보자>

input 은 관련된 모든 주의 하루 % + 코스피 % - https://finance.naver.com/item/sise_day.nhn?code=005930&page=1

output 은 그중 하나의 다음날 %


% 받아오는 모듈을 만들어 보자

In [50]:
from selenium import webdriver
from bs4 import BeautifulSoup

############################################################### for headless options
option = webdriver.ChromeOptions()
option.add_argument('headless')
option.add_argument('window-size=1920x1080')
option.add_argument("disable-gpu")
driver=webdriver.Chrome('C:/Users/mod96/Desktop/HSH/Programs/WebCrawling/chromedriver', options=option)
###############################################################
def Percentmodule(code,endpage): # 둘다 str 형식. 한 페이지에 10개.
    PM=[];Price=[];Percent=[]
    for i in range(int(endpage)):
        pagenum=str(i+1)
        URL = 'https://finance.naver.com/item/sise_day.nhn?code='+code+'&page='+pagenum
        driver.get(URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ############################################ PM 은 전일비 가격 변동
        main=soup.select("span.tah.p11.red02,span.tah.p11.nv01")
        for m in main:
            if str(m).find('nv01')>0:
                PM.append(-1*int(str(m.text).replace('\t','').replace('\n','').replace(',','')))
            else:
                PM.append(int(str(m.text).replace('\t','').replace('\n','').replace(',','')))
        ############################################ Price 는 당일 종가
        end=soup.select("td.num")
        M=[]
        for e in end:
            M.append(e.text)
        k=0
        while 6*k<len(M):
            Price.append(int(str(M[6*k]).replace('\t','').replace('\n','').replace(',','')))
            k=k+1
        ############################################ 당일 PM/전날 Price = 증감율
    for i in range(len(PM)-1):
        Percent.append(round(100*PM[i]/Price[i+1],3))
    Percent.reverse()
    return Percent

###############################################################
Percent=Percentmodule('005930','2')
for i in range(len(Percent)):
    print(Percent[i])

1.937
-2.418
-1.062
-3.041
1.476
0.727
3.61
0.697
-2.249
-4.071
0.738
-4.579
-2.495
-1.673
-2.102
-3.272
-3.594
-5.811
5.704


주요 지표들 url
===========
1. 환율 시세
---------------페이지당 10개
USD : 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW&page=1'\\
EUR : 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_EURKRW&page=1'\\
JPY : 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_JPYKRW&page=1'\\
CNY : 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_CNYKRW&page=1'

IDX1=['USD','EUR','JPY','CNY']\\
IDX2=['GSL','HGSL','LO']\\
IDX3=['GC','SI','PA']\\
IDX4=['020150','336370','098460','084850','011070','009150','028260','006400','018260','207940','005930','066570','000660','082270','039030','036830','005935','240810','000990']

2. 자원 시세
---------------페이지당 7개
휘발유 : 
'https://finance.naver.com/marketindex/oilDailyQuote.nhn?marketindexCd=OIL_GSL&page=1'\\
고급휘발유 : 
'https://finance.naver.com/marketindex/oilDailyQuote.nhn?marketindexCd=OIL_HGSL&page=1'\\
경유 : 
'https://finance.naver.com/marketindex/oilDailyQuote.nhn?marketindexCd=OIL_LO&page=1'\\
국제금 : 
'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_GC&fdtc=2&page=1'\\
은 : 
'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_SI&fdtc=2&page=1'\\
팔라듐 : 
'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_PA&fdtc=2&page=1'\\

3. 주가
전자기기 : ['020150','336370','098460','084850','011070','009150']\\
삼성그룹 : ['028260','006400','018260','207940']\\
전자,반도체(대기업) : ['005930','066570','000660']\\
반도체(시총기준) : ['082270','039030','036830','005935','240810','000990']

합쳐봅시다.


In [20]:
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
############################################################### for headless options
option = webdriver.ChromeOptions()
option.add_argument('headless')
option.add_argument('window-size=1920x1080')
option.add_argument("disable-gpu")
driver=webdriver.Chrome('C:/Users/mod96/Desktop/HSH/Programs/WebCrawling/chromedriver', options=option)
#############################################################################################################################
def Percentmodule1(code,endpage): # 둘다 str 형식. 한 페이지에 10개.
    PM=[];Price=[];Percent=[]
    for i in range(int(endpage)):
        pagenum=str(i+1)
        URL = 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_'+code+'KRW&page='+pagenum
        driver.get(URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ############################################ PM 은 전일비 가격 변동
        main=soup.select("td.num")
        for m in main:
            if str(m).find('https://ssl.pstatic.net/static/nfinance/ico_down.gif')>0:
                PM.append(-1*float(str(m.text).replace('\t','').replace('\n','').replace(',','').replace(' ','')))
            elif str(m).find('https://ssl.pstatic.net/static/nfinance/ico_up.gif')>0:
                PM.append(float(str(m.text).replace('\t','').replace('\n','').replace(',','').replace(' ','')))
            elif str(m).find('https://ssl.pstatic.net/static/nfinance/ico_same2.gif')>0:
                PM.append(0)
            else:
                Price.append(float(str(m.text).replace('\t','').replace('\n','').replace(',','').replace(' ','')))
    for i in range(len(PM)-1):
        Percent.append(round(100*PM[i]/Price[i+1],3))
    Percent.reverse()
    return Percent
#############################################################################################################################
def Percentmodule2(code,endpage): # 둘다 str 형식. 한 페이지에 10개.
    Percent=[]
    for i in range(int(endpage)):
        pagenum=str(i+1)
        URL = 'https://finance.naver.com/marketindex/oilDailyQuote.nhn?marketindexCd=OIL_'+code+'&page='+pagenum
        driver.get(URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ############################################ PM 은 전일비 가격 변동
        main=soup.select("td.num")
        for m in main:
            if str(m).find('%')>0:
                Percent.append(float(str(m.text).replace('\t','').replace('\n','').replace(',','').replace('%','').replace(' ','')))
    Percent.reverse()
    del Percent[0]
    return Percent
#############################################################################################################################
def Percentmodule3(code,endpage): # 둘다 str 형식. 한 페이지에 10개.
    Percent=[]
    for i in range(int(endpage)):
        pagenum=str(i+1)
        URL = 'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_'+code+'&fdtc=2&page='+pagenum
        driver.get(URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ############################################ PM 은 전일비 가격 변동
        main=soup.select("td.num")
        for m in main:
            if str(m).find('%')>0:
                Percent.append(float(str(m.text).replace('\t','').replace('\n','').replace(',','').replace('%','').replace(' ','')))
    Percent.reverse()
    del Percent[0]
    return Percent
#############################################################################################################################
def Percentmodule4(code,endpage): # 둘다 str 형식. 한 페이지에 10개.
    Price=[];Percent=[]
    for i in range(int(endpage)):
        pagenum=str(i+1)
        URL = 'https://finance.naver.com/item/sise_day.nhn?code='+code+'&page='+pagenum
        driver.get(URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ############################################ 당일 PM/전날 Price = 증감율
        main=soup.select("td.num")
        M=[]
        for e in main:
            M.append(e.text)
        k=0
        while 6*k<len(M):
            Price.append(float(str(M[6*k]).replace('\t','').replace('\n','').replace(',','').replace(' ','')))
            k=k+1
    for i in range(len(Price)-1):
        Percent.append(round(100*(Price[i]-Price[i+1])/Price[i+1],3))
    Percent.reverse()
    return Percent
###############################################################
IDX1=['USD','EUR','JPY','CNY']  #환율
IDX2=['GSL','HGSL','LO']  #유가
IDX3=['GC','SI','PA']  #금은팔
IDX4=['020150','098460','011070','009150','028260','006400','018260','207940','066570','000660','082270','039030','036830','005935','240810','000990','005930']
l=len(IDX1)+len(IDX2)+len(IDX3)+len(IDX4)
endpage1=77 #페이지당 10개
endpage2=110 #페이지당 7개
#820개 아래로 => 770개

A=[]
for code in IDX1:
    A.append(Percentmodule1(code,endpage1))
for code in IDX2:
    A.append(Percentmodule2(code,endpage2))
for code in IDX3:
    A.append(Percentmodule3(code,endpage2))
for code in IDX4:
    A.append(Percentmodule4(code,endpage1))
    
import csv
file=open("C:/Users/mod96/Desktop/HSH/data.csv",'w',newline="")
csvwriter=csv.writer(file)
for row in A:
    csvwriter.writerow(row)
file.close()

import winsound as ws
print(ws.Beep(2000, 1000))

None


In [16]:
for i in range(len(A)):
    print(len(A[i]))

769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769


In [18]:
import winsound as ws
print(ws.Beep(2000, 1000))

None


In [7]:
820/7

117.14285714285714

최종페이지 확인하는 모듈
=>[100, 100, 100, 100, 100, 100, 100, 83, 100, 100, 100, 100, 100, 100, 96, 100, 100]

In [29]:
def whatpage(code,maxpage):
    T=True;i=0;
    while T==True and i<maxpage:
        pagenum=str(i+1)
        URL = 'https://finance.naver.com/item/sise_day.nhn?code='+code+'&page='+pagenum
        driver.get(URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ############################################ PM 은 전일비 가격 변동
        main=soup.select("td.num")
        M=[]
        for e in main:
            M.append(e.text)
        k=0
        while 6*k<len(M):
            if str(M[6*k])=='\xa0':
                T=False;
                break;
            k=k+1
        i=i+1
    return i
maxpage=100
PAGE=[]
IDX4=['020150','098460','084850','011070','009150','028260','006400','018260','207940','066570','000660','082270','039030','036830','005935','240810','000990','005930']
for code in IDX4:
    PAGE.append(whatpage(code,maxpage))    
print(PAGE)

[100, 100, 10, 100, 100, 100, 100, 100, 83, 100, 100, 100, 100, 100, 100, 96, 100, 100]


csv 저장부터

In [33]:
A=[[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
import csv

file=open("C:/Users/mod96/Desktop/HSH/data.csv",'w',newline="")
csvwriter=csv.writer(file)
for row in A:
    csvwriter.writerow(row)
file.close()

In [6]:

IDX1=['USD','EUR','JPY','CNY']  #환율
IDX2=['GSL','HGSL','LO']  #유가
IDX3=['GC','SI','PA']  #금은팔
IDX4=['020150','336370','098460','084850','011070','009150','028260','006400','018260','207940','066570','000660','082270','039030','036830','005935','240810','000990','005930']

l=len(IDX1)+len(IDX2)+len(IDX3)+len(IDX4)
print(l)

29


In [35]:
import tensorflow as tf
import numpy

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, input_dim=l-1, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')    
    ])

model.compile(loss='mean_squared_error',
              optimizer='adam', metrics=['accuracy'])

In [36]:
X=A[:len(A)-1][:len(A[0])-1]
Y=A[len(A)-1][1:]
model.fit(X, Y, epochs=10)
model.evaluate(X,  Y, verbose=2)

ValueError: Error when checking input: expected dense_input to have shape (26,) but got array with shape (1,)

Version 03.25. - 크롤링 완료. 러닝을 고차원화 해야함.